# 📘 Notebook 01 — EDA & Preparação do Dataset (Renda x Imóveis x Dívidas)

Projeto: **Clusterização em 3D com K-Means**  
Disciplina: **Aprendizado de Máquina Não Supervisionado** — Senac DF  
Autores: **Anderson de Matos Guimarães, Renan Ost, Gustavo Stefano Thomazinho**

**Objetivo deste notebook**  
Explorar o dataset bruto de **Distribuição de Renda por Centis** (IRPF), entender o negócio, selecionar **3 variáveis contínuas** e produzir um dataset **limpo e padronizado** para o Notebook 02 (clusterização e visualização 3D).

## 🎯 Escopo & Entregáveis

**Vamos:**
1. Carregar e validar o CSV bruto.  
2. Inspecionar colunas, tipos, nulos, duplicatas e consistência.  
3. Documentar o **dicionário de dados** (visão de negócio).  
4. Definir a **amostragem reprodutível** (300–500 linhas) com critérios claros.  
5. Selecionar e preparar as **3 variáveis**:
   - `rtb_soma_centil` (renda),
   - `bens_imoveis` (patrimônio),
   - `dividas_onus` (endividamento).
6. Tratar outliers e zeros estruturais quando necessário (sem distorcer a realidade).  
7. Escalonar (opcional) e **salvar** o dataset tratado (+ `metadata.json`).  

**Saídas:**
- `data/processed/distribuicao-renda-3vars.csv`  
- `data/processed/distribuicao-renda-3vars.metadata.json`  
- Gráficos e anotações que justificam as decisões.

## ✅ Critérios do Professor (como atendemos)

- **Entradas (300–500)** → definiremos uma **amostra reprodutível** com base em (ano, entes federativos, centis).  
- **Dados granulares** → centis (100 cortes por distribuição de RTB) garantem granularidade.  
- **Numéricos contínuos** → valores monetários (R$) para as 3 variáveis.  
- **Exatamente 3 variáveis** → renda, patrimônio (imóveis) e dívidas (3D pronto).  
- **Notebook estilo IDEB** → manteremos seções claras, decisões justificadas e, no 02, poderemos comparar diferentes *k* (e opcionalmente usar dois anos se for relevante).

## 🗂️ Fonte de Dados, Licença e Paths

- **Fonte oficial**: Receita Federal — Distribuição de Renda por Centis.  
- **Arquivo bruto**: `data/raw/distribuicao-renda.csv`  
- **Arquivo tratado (3 variáveis)**: `data/processed/distribuicao-renda-3vars.csv`

> Observação: trabalharemos com unidades conforme o arquivo (muitos campos estão em **R$ milhões**). Faremos padronização de nomes e registraremos as unidades no metadado.

## 🏢 Entendimento do Negócio (resumo)

O dataset agrega informações de declarações de **IRPF** por **centis de renda tributável bruta (RTB)**.  
Cada linha representa um **grupo** (centil) para um **ente federativo** em um **ano**.  
As colunas trazem somatórios de rendimentos, bens/direitos, despesas dedutíveis, dívidas, etc.

**Hipóteses de leitura econômica (para orientar a análise):**
- `rtb_soma_centil` aproxima **capacidade de geração de renda** do grupo.  
- `bens_imoveis` aproxima **acumulação patrimonial** estável.  
- `dividas_onus` aproxima **alavancagem/endividamento**.  

Essas três dimensões juntas formam um **espaço 3D** que deveria segregar perfis de grupos (baixa renda com baixa riqueza e baixa dívida vs. alta renda com alta riqueza e dívida variável, etc.).

## 📖 Dicionário de Dados (campos relevantes ao projeto)

| Coluna original (exemplo)                 | Nome padronizado        | Tipo      | Unidade         | Observação de negócio |
|-------------------------------------------|-------------------------|-----------|-----------------|-----------------------|
| Ano-calendário                            | `ano`                   | int       | ano             | Ano da declaração     |
| Ente Federativo                           | `uf`                    | string    | —               | Estado/Agregado       |
| Centil                                    | `centil`                | float     | 1–100           | Corte por RTB         |
| Rend. Trib. — Soma da RTB do Centil       | `rtb_soma_centil`       | float     | R$ milhões      | **Renda** (capacidade)|
| Bens e Direitos — Imóveis                 | `bens_imoveis`          | float     | R$ milhões      | **Patrimônio**        |
| Dívidas e Ônus                            | `dividas_onus`          | float     | R$ milhões      | **Endividamento**     |

> Notas:
> - Confirmaremos nomes exatos das colunas do CSV bruto e mapearemos para os padronizados acima.  
> - Se necessário, convertendo vírgulas decimais e removendo formatações.

In [ ]:
# Imports, seed e paths (corrigidos para notebook em /notebooks)
from __future__ import annotations
import os, json, math, textwrap, re, sys
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
pd.set_option("display.max_columns", 120)
pd.set_option("display.float_format", lambda v: f"{v:,.3f}")

# Paths
ROOT = Path("..").resolve()  # sobe um nível a partir de /notebooks
DATA_RAW = ROOT / "data" / "raw" / "distribuicao-renda.csv"
DATA_PROCESSED_DIR = ROOT / "data" / "processed"
DATA_PROCESSED = DATA_PROCESSED_DIR / "distribuicao-renda-3vars.csv"
METADATA = DATA_PROCESSED_DIR / "distribuicao-renda-3vars.metadata.json"
FIG_DIR = ROOT / "reports" / "figures"
for p in [DATA_PROCESSED_DIR, FIG_DIR]:
    p.mkdir(parents=True, exist_ok=True)

FALLBACK_FILE = Path("/mnt/data/distribuicao-renda.csv")
if not DATA_RAW.exists() and FALLBACK_FILE.exists():
    print(f"[INFO] Usando fallback: {FALLBACK_FILE}")
    DATA_RAW = FALLBACK_FILE

print("ROOT        :", ROOT)
print("DATA_RAW    :", DATA_RAW)
print("PROCESSED   :", DATA_PROCESSED)
print("FIG_DIR     :", FIG_DIR)

In [ ]:
def read_csv_br(filepath: Path) -> pd.DataFrame:
    """
    Leitura robusta para CSV com possíveis variações:
    - separador ',' ou ';'
    - decimal '.' ou ','
    - encoding 'utf-8' ou 'latin-1'
    Retorna o DataFrame lido com detecção automática.
    """
    trials = [
        dict(sep=";", decimal=",", encoding="utf-8", engine="python"),
        dict(sep=",", decimal=",", encoding="utf-8", engine="python"),
        dict(sep=";", decimal=".", encoding="utf-8", engine="python"),
        dict(sep=",", decimal=".", encoding="utf-8", engine="python"),
        dict(sep=";", decimal=",", encoding="latin-1", engine="python"),
        dict(sep=",", decimal=",", encoding="latin-1", engine="python"),
    ]
    last_err = None
    for opts in trials:
        try:
            df = pd.read_csv(filepath, **opts)
            if df.shape[1] == 1:
                last_err = RuntimeError("provável separador incorreto (1 coluna)")
                continue
            print(f"[OK] Leitura com parâmetros: {opts}")
            return df
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Falha ao ler {filepath}: {last_err}")

def snake(s: str) -> str:
    s2 = re.sub(r"[^\w]+", "_", s.strip().lower(), flags=re.UNICODE)
    s2 = re.sub(r"_{2,}", "_", s2).strip("_")
    return s2

def find_col(candidates: List[str], patterns: List[str]) -> str | None:
    for c in candidates:
        cl = c.lower()
        ok = True
        for pat in patterns:
            ors = pat.split("|")
            if not any(o in cl for o in ors):
                ok = False
                break
        if ok:
            return c
    return None

def quantiles_report(s: pd.Series, qs=(0.5, 0.9, 0.95, 0.99)) -> pd.Series:
    qv = s.quantile(q=list(qs))
    qv.index = [f"q{int(q*100):02d}" for q in qs]
    return qv

def savefig(path: Path):
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    print(f"[FIG] salvo: {path}")

## 🔍 EDA do Arquivo Bruto

Nesta seção faremos:
1. **Leitura segura** (encoding, separador, decimal).  
2. **Shape, colunas, tipos, nulos, duplicatas**.  
3. **Estatísticas descritivas** (mediana, p95, p99) para entender caudas.  
4. **Sanidade de chaves lógicas**: (ano, uf, centil) sem duplicidades por registro.  
5. **Distribuições**:
   - Histogramas de `rtb_soma_centil`, `bens_imoveis`, `dividas_onus`.  
   - Scatterpairs para relações bivariadas.  
6. **Zeros e “ausências esperadas”**: bens ou dívidas podem ter zeros; não confundir com *missing*.  
7. **Outliers**: avaliar se são fenômeno real (altíssima concentração no topo) → provavelmente **não remover**, apenas documentar e considerar **log-transform** opcional.

In [ ]:
df_raw = read_csv_br(DATA_RAW)

print("\n# VISÃO GERAL")
print("shape:", df_raw.shape)
display(df_raw.head(10))

print("\n# COLUNAS")
print(list(df_raw.columns))

print("\n# TIPOS")
display(df_raw.dtypes)

print("\n# NULOS (%)")
display((df_raw.isna().mean() * 100).round(2).sort_values(ascending=False))

print("\n# Duplicatas (linhas idênticas):", df_raw.duplicated().sum())

In [ ]:
cols = list(df_raw.columns)

col_ano    = find_col(cols, ["ano", "calendario|calendário"])
col_uf     = find_col(cols, ["ente|uf|federativo"])
col_centil = find_col(cols, ["centil"])

col_rtb_soma   = find_col(cols, ["rendimentos|rtb", "soma|somatorio|somatório"])
col_bens_imov  = find_col(cols, ["bens|direitos", "imoveis|imóveis"])
col_dividas    = find_col(cols, ["dividas|dívidas", "onus|ônus"])

mapping = {
    col_ano: "ano",
    col_uf: "uf",
    col_centil: "centil",
    col_rtb_soma: "rtb_soma_centil",
    col_bens_imov: "bens_imoveis",
    col_dividas: "dividas_onus",
}

print("# MAPEAMENTO PROPOSTO")
display(mapping)

missing_keys = [k for k in mapping if k is None]
if missing_keys:
    raise ValueError("Não consegui detectar automaticamente algumas colunas. Revise os padrões desta célula e rode novamente.")

df = df_raw.rename(columns={k: v for k, v in mapping.items()})
df.columns = [snake(c) for c in df.columns]
display(df.head(3))

In [ ]:
# === Tipagem robusta ===
df["ano"] = (
    df["ano"].astype(str).str.extract(r"(\d{4})", expand=False)
      .pipe(pd.to_numeric, errors="coerce")
      .astype("Int64")
)

centil_raw = (
    df["centil"]
      .astype(str)
      .str.replace(",", ".", regex=False)
      .str.extract(r"(\d+(?:\.\d+)?)", expand=False)
)
df["centil"] = pd.to_numeric(centil_raw, errors="coerce")

for c in ["rtb_soma_centil", "bens_imoveis", "dividas_onus"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=["rtb_soma_centil", "bens_imoveis", "dividas_onus"])

mask_centil = (df["centil"] >= 1) & (df["centil"] <= 100)
rem_out = (~mask_centil).sum()
if rem_out:
    print(f"[INFO] Removendo {rem_out} linhas com centil fora de [1,100]")
df = df[mask_centil].copy()

dup = df.duplicated(subset=["ano", "uf", "centil"]).sum()
print("Duplicidades em (ano, uf, centil):", dup)

print("Shape após limpeza básica:", df.shape)
display(df[["ano", "uf", "centil", "rtb_soma_centil", "bens_imoveis", "dividas_onus"]].head())

In [ ]:
num_cols = ["rtb_soma_centil", "bens_imoveis", "dividas_onus"]
display(df[num_cols].describe().T)

qtab = pd.concat([quantiles_report(df[c]) for c in num_cols], axis=1).T
qtab.columns = qtab.columns.str.upper()
display(qtab)

print("anos:", sorted(df["ano"].dropna().unique().tolist())[:10], "...")
print("UFs (amostra):", df["uf"].dropna().unique()[:10])
print("linhas totais:", len(df))

In [ ]:
plt.figure(figsize=(12,3.6))
for i,c in enumerate(num_cols, 1):
    plt.subplot(1,3,i)
    plt.hist(df[c].dropna(), bins=40)
    plt.title(c)
    plt.xlabel("valor")
    plt.ylabel("freq")
plt.tight_layout()
plt.show()

pairs = [("rtb_soma_centil","bens_imoveis"),
         ("rtb_soma_centil","dividas_onus"),
         ("bens_imoveis","dividas_onus")]

plt.figure(figsize=(12,3.6))
for i,(x,y) in enumerate(pairs,1):
    plt.subplot(1,3,i)
    plt.scatter(df[x], df[y], s=8, alpha=0.6)
    plt.xlabel(x); plt.ylabel(y)
    plt.title(f"{x} vs {y}")
plt.tight_layout()
plt.show()

for c in num_cols:
    df[f"log_{c}"] = np.log1p(df[c].clip(lower=0))

## 🧪 Estratégia de Amostragem (300–500 linhas)

**Princípio**: reprodutibilidade + representatividade.

**Passos**:
1. Escolher **um ano-base** (tipicamente o mais recente disponível).  
2. Calcular o total de linhas (UF × centis) e **estimar** quantos UFs e centis precisamos para cair entre **300–500**.  
3. Estratégias possíveis (a definir após ver o shape real):
   - **E1 (recomendada)**: Fixar ano; **usar todos os UFs**; selecionar um **intervalo contínuo de centis** (ex.: 1–20, 30–60, 90–100) que dê ~300–500.  
   - **E2**: Fixar ano; **amostrar UFs** (estratificado por região) e usar **todos os centis** desses UFs.  
4. Fixar uma **seed** para qualquer amostragem aleatória.  
5. Registrar a regra no `metadata.json`.

> Justificativa didática: manter **centis contíguos** preserva estrutura da distribuição e facilita interpretar clusters (baixa, média, alta renda).

In [ ]:
def choose_year_base(data: pd.DataFrame) -> int:
    anos = data["ano"].dropna().astype(int)
    year = int(anos.max())
    print(f"[ANO-BASE] Selecionado automaticamente: {year}")
    return year

def sample_rows(data: pd.DataFrame, target_min: int = 300, target_max: int = 500) -> pd.DataFrame:
    year = choose_year_base(data)
    dfy = data.query("ano == @year").copy()
    possiveis_agregados = {"brasil", "nacional", "todos", "agregado"}
    dfy["uf_lc"] = dfy["uf"].astype(str).str.lower()
    dfy = dfy[~dfy["uf_lc"].isin(possiveis_agregados)].drop(columns=["uf_lc"])

    n_uf = dfy["uf"].nunique()
    print(f"[AMOSTRA] UFs distintos no ano {year}: {n_uf}")

    N_min = math.ceil(target_min / n_uf)
    N_max = min(100, math.floor(target_max / n_uf))
    N = max(1, min(N_max, max(N_min, 10)))
    print(f"[AMOSTRA] Intervalo de centis: 1..{N} (alvo {target_min}-{target_max})")

    df_sample = dfy[dfy["centil"].between(1, N, inclusive="both")].copy()
    print(f"[AMOSTRA] Linhas resultantes: {len(df_sample)}")
    return df_sample

df_sample = sample_rows(df)
display(df_sample.head())
display(df_sample.tail())
print("shape:", df_sample.shape)

In [ ]:
dups = df_sample.duplicated(subset=["ano", "uf", "centil"]).sum()
print("Duplicidades (ano,uf,centil) na amostra:", dups)

print("Nulos nas variáveis selecionadas:")
display(df_sample[["rtb_soma_centil", "bens_imoveis", "dividas_onus"]].isna().sum())

display(df_sample[["rtb_soma_centil", "bens_imoveis", "dividas_onus"]].describe().T)

## 🧼 Limpeza & Transformações

1. **Padronizar nomes** de colunas para *snake_case*.  
2. **Selecionar colunas**: `ano`, `uf`, `centil`, `rtb_soma_centil`, `bens_imoveis`, `dividas_onus`.  
3. **Tipos corretos** (int/float); tratar decimal com vírgula, se houver.  
4. **Checagens de integridade**:
   - `centil` ∈ [1, 100]  
   - Sem duplicidade para (ano, uf, centil) na amostra.  
5. **Tratamento de escalas**:
   - Manter valores em **R$ milhões** (consistência com a fonte).  
   - Criar **versão log-transform** para visualização, se necessário (`log1p`).  
6. **Escalonamento (para o Notebook 02)**:
   - Salvar **dados crus** e, opcionalmente, uma **cópia escalada** (StandardScaler/MinMax) para K-Means.  
   - A decisão final de escalonamento será aplicada no Notebook 02; aqui apenas deixamos a função e um preview.

In [ ]:
cols_final = ["ano", "uf", "centil", "rtb_soma_centil", "bens_imoveis", "dividas_onus"]
df_out = df_sample[cols_final].copy()

# Salvar
DATA_PROCESSED.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(DATA_PROCESSED, index=False)
print(f"[SALVO] {DATA_PROCESSED} ({len(df_out)} linhas)")

# Metadados
metadata = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "source_file": str(DATA_RAW),
    "output_file": str(DATA_PROCESSED),
    "year_base": int(df_out["ano"].dropna().max()) if len(df_out) else None,
    "sampling_rule": "ano=max; UFs=all (excl. agregados nacionais); centis=1..N tal que linhas entre 300-500",
    "n_rows": int(len(df_out)),
    "units": {
        "rtb_soma_centil": "R$ milhões (somatório por centil)",
        "bens_imoveis": "R$ milhões (somatório por UF-centil)",
        "dividas_onus": "R$ milhões (somatório por UF-centil)",
    },
    "notes": [
        "Colunas padronizadas para snake_case.",
        "Zeros podem representar ausência real de bens/dívidas.",
        "EDA completa salva em reports/figures/.",
        "Transformações log(1+x) usadas apenas para visualização na EDA.",
    ],
    "random_seed": RANDOM_SEED,
    "authors": [
        "Anderson de Matos Guimarães",
        "Renan Ost",
        "Gustavo Stefano Thomazinho",
    ],
}
import json
with open(METADATA, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)
print(f"[SALVO] {METADATA}")

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X = df_out[["rtb_soma_centil", "bens_imoveis", "dividas_onus"]].values

sc_std = StandardScaler()
X_std = sc_std.fit_transform(X)

sc_mm = MinMaxScaler()
X_mm = sc_mm.fit_transform(X)

print("Preview StandardScaler (primeiras 5 linhas):")
print(pd.DataFrame(X_std, columns=["rtb_std","imoveis_std","dividas_std"]).head())

print("\nPreview MinMaxScaler (primeiras 5 linhas):")
print(pd.DataFrame(X_mm, columns=["rtb_mm","imoveis_mm","dividas_mm"]).head())

## 💾 Persistência dos Resultados

- **CSV final**: `data/processed/distribuicao-renda-3vars.csv`  
  - Colunas: `ano, uf, centil, rtb_soma_centil, bens_imoveis, dividas_onus`  
  - Somente linhas da **amostra definida**.  
- **Metadados**: `data/processed/distribuicao-renda-3vars.metadata.json`  
  - `created_at`, `source_file`, `year_base`, `sampling_rule`, `n_rows`, `units`, `notes`.  
- **Imagens** (opcional): `reports/figures/eda_*`

## 🧰 Reprodutibilidade e Versão

- Scripts utilitários em `src/utils.py` (funções de leitura, checagens, gráficos rápidos).  
- Fixar `RANDOM_SEED` no topo do notebook.  
- Salvar `pip freeze` (opcional) em `requirements.txt` (já existe no repo).  
- Comentar decisões no corpo do notebook para facilitar a correção.

## ⚖️ Limitações & Ética

- Dados **agregados** por centis (não individuais).  
- Possível **assimetria extrema** nos top centis (riqueza concentrada).  
- “Zero” em patrimônio/dívida pode indicar **ausência real**, não erro.  
- Interpretações devem ser **econômicas** e **contextualizadas** (não normativas).

## 🗺️ Próximos Passos (Notebook 02)

- Escolha do **k** (Elbow, Silhouette).  
- **K-Means** com dados escalados.  
- **Gráfico 3D interativo** (Plotly) dos clusters.  
- **Animação** (Formação dos clusters / frames por iteração ou por *k*).  
- Interpretação dos grupos e relato.

## ✅ Checklist (para eu mesmo)

- [ ] CSV bruto carregado e validado  
- [ ] Dicionário de dados preenchido com nomes exatos  
- [ ] Estratégia de amostragem definida e aplicada  
- [ ] 3 variáveis selecionadas e conferidas  
- [ ] CSV tratado salvo + metadados gerados  
- [ ] Gráficos EDA salvos em `reports/figures/`  
- [ ] Commit com mensagem padrão **conventional commits**

In [ ]:
checks = {
    "csv_tratado_existe": DATA_PROCESSED.exists(),
    "metadata_existe": METADATA.exists(),
    "linhas_entre_300_500": (300 <= len(pd.read_csv(DATA_PROCESSED)) <= 500) if DATA_PROCESSED.exists() else False,
    "tem_colunas_certas": (set(pd.read_csv(DATA_PROCESSED).columns) == {"ano", "uf", "centil", "rtb_soma_centil", "bens_imoveis", "dividas_onus"}) if DATA_PROCESSED.exists() else False,
}
print(checks)
print("[OK] Todos os checks passaram." if all(checks.values()) else "[ATENÇÃO] Algum check falhou.")